## Library Importation

In [1]:
import pandas as pd
import plotly.express as px

In [2]:
# Load datasets
apple_store = pd.read_excel("AppleStore.xlsx", index_col="id", engine="openpyxl")
description = pd.read_excel("appleStore_description.xlsx", engine="openpyxl")
apple_store.drop(columns="Unnamed: 0", inplace=True)

## Data Preprocessing:

During the data preprocessing steps, it was observed that some entries in the dataset had 0 in the `lang_num` column, indicating that the number of supported languages for these apps was not available or known. To enhance clarity and handle these cases, the entries with 0 in `lang_num` have been replaced with "N/A."

This replacement is made to signify that the information about the supported languages is not provided, rather than indicating that the app supports zero languages. As a result, when encountering "N/A" in the `lang_num` column, it implies that the data for the number of languages may be missing or unspecified.

In [3]:
# Merge the two datasets
merged_data = pd.merge(apple_store, description, on=['id', 'track_name', 'size_bytes'], how='left')
merged_data.set_index('id', inplace=True)

In [4]:
# replace 0 in the lang_num column with N/A
merged_data["lang_num"].replace(0, "N/A", inplace=True)

## Feature Engineering

In [5]:
# Convert size in bytes to megabytes
merged_data['size_MB'] = merged_data['size_bytes'] / (1024 ** 2)

In [8]:
# classifying each app as free or paid

# Ensure 'free_paid' column is of an appropriate data type (e.g., object)
merged_data['free_paid'] = merged_data['price'].astype('object')

# Iterate through the rows and set the 'free_paid' column based on the price
for index, row in merged_data.iterrows():
    if row['price'] == 0.00:
        merged_data.loc[index, 'free_paid'] = "Free"
    else:
        merged_data.loc[index, 'free_paid'] = "Paid"

In [ ]:
# grouping the number of languages supported by the app

# Ensure 'lang_num' column is of an appropriate data type (e.g., Int64)
merged_data['lang_num'] = pd.to_numeric(merged_data['lang_num'], errors='coerce').astype('Int64')

bins = [0, 20, 40, 60, float('inf')]
labels = ['0-20', '20-40', '40-60', '60+']

# Create a new column 'lang_num_group' using pd.cut()
merged_data['lang_num_group'] = pd.cut(merged_data['lang_num'], bins=bins, labels=labels, right=False)

In [ ]:
# grouping the siz in MB of the app

# Define the size ranges
bins = [0, 500, 1000, 1500, 2000, 2500, 3000, float('inf')]
labels = ["0-500 MB", "500-1000 MB", "1000-1500 MB", "1500-2000 MB", "2000-2500 MB", "2500-3000 MB", "3000+ MB"]

# Create a new column 'size_group' using pd.cut()
merged_data['size_group'] = pd.cut(merged_data['size_MB'], bins=bins, labels=labels, right=False)


In [ ]:
# Define the price ranges
bins = [0, 0.1, 50, 100, 150, 200, float('inf')]
labels = ['free', '0.1-50 dollars', '50-100 dollars', '100-150 dollars', '150-200 dollars', 'Above 200 dollars']

# Create a new column 'price_group' using pd.cut()
merged_data['price_group'] = pd.cut(merged_data['price'], bins=bins, labels=labels, right=False)

In [ ]:
# merged_data.to_excel('FinalAppleData.xlsx')

In [ ]:
merged_data.to_csv("FinalAppleData.csv")